In [94]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from dateutil.relativedelta import relativedelta
import calendar
from utils import *

In [91]:
# Data and super variables
# Import the data for fund 1
f1_DailyAuM = pd.read_csv("./Data/Daily AuM - AuM (1).csv")
f1_DailyTrades = pd.read_csv("./Data/Daily AuM - Trades_10.csv")

# Get the union of both datasets (Date-wise)
f1_DailyAuM.Date = pd.to_datetime(f1_DailyAuM.Date, dayfirst = True, format = "mixed")
f1_DailyTrades.buy_date = pd.to_datetime(f1_DailyTrades.buy_date, dayfirst = True, format = "mixed")
f1_DailyTrades.sell_date = pd.to_datetime(f1_DailyTrades.sell_date, dayfirst = True, format = "mixed")

f1_DailyTrades_LastDate = f1_DailyTrades.iloc[-1].sell_date
f1_DailyAuM_LastDate = f1_DailyAuM.iloc[-1].Date

if f1_DailyTrades_LastDate < f1_DailyAuM_LastDate:
    f1_DailyAuM = f1_DailyAuM[f1_DailyAuM.Date <= f1_DailyTrades_LastDate]
else:
    f1_DailyTrades = f1_DailyTrades[f1_DailyTrades.sell_date <= f1_DailyAuM_LastDate]

# start and end of date of the entire dataset
START_DATE, END_DATE = f1_DailyAuM.iloc[0].Date, f1_DailyAuM.iloc[-1].Date 
START_MONTH, END_MONTH = f1_DailyAuM.iloc[0].Date.month, f1_DailyAuM.iloc[-1].Date.month 
START_YEAR, END_YEAR = f1_DailyAuM.iloc[0].Date.year, f1_DailyAuM.iloc[-1].Date.year 

print(f"The entire date range is from {START_DATE} to {END_DATE} ({(f1_DailyAuM.iloc[-1].Date - f1_DailyAuM.iloc[0].Date)})")


The entire date range is from 2019-12-27 00:00:00 to 2023-10-31 00:00:00 (1404 days 00:00:00)


In [92]:
# Growth rate of fund net worth
# This metric is calculated on three intervals, six months, 1 year and 3 years
range_6Mo = divideDateRange(START_DATE, END_DATE, datetime.timedelta(days=30*6))
range_1Yr = divideDateRange(START_DATE, END_DATE, datetime.timedelta(days=30*12))
range_3Yrs = divideDateRange(START_DATE, END_DATE, datetime.timedelta(days=30*12*3))

tmpAuM_6Mo = f1_DailyAuM[(range_6Mo[1]<f1_DailyAuM.Date)&(f1_DailyAuM.Date<=range_6Mo[0])]
growthRate_6Mo = netGrowthRate(tmpAuM_6Mo.iloc[-1].AuM, tmpAuM_6Mo.iloc[0].AuM, False)

tmpAuM_1Yr = f1_DailyAuM[(range_1Yr[1]<f1_DailyAuM.Date)&(f1_DailyAuM.Date<=range_1Yr[0])]
growthRate_1Yr = netGrowthRate(tmpAuM_1Yr.iloc[-1].AuM, tmpAuM_1Yr.iloc[0].AuM, False)

tmpAuM_3Yrs = f1_DailyAuM[(range_3Yrs[1]<f1_DailyAuM.Date)&(f1_DailyAuM.Date<=range_3Yrs[0])]
growthRate_3Yrs = netGrowthRate(tmpAuM_3Yrs.iloc[-1].AuM, tmpAuM_3Yrs.iloc[0].AuM, False)

print(f"6 months growth rate (%) = {growthRate_6Mo*100} calculated from {tmpAuM_6Mo.iloc[-1].Date} to {tmpAuM_6Mo.iloc[0].Date}")
print(f"1 year growth rate (%) = {growthRate_1Yr*100} calculated from {tmpAuM_1Yr.iloc[-1].Date} to {tmpAuM_1Yr.iloc[0].Date}")
print(f"3 years growth rate (%) = {growthRate_3Yrs*100} calculated from {tmpAuM_3Yrs.iloc[-1].Date} to {tmpAuM_3Yrs.iloc[0].Date}")


6 months growth rate (%) = 4.3824640703169635 calculated from 2023-10-31 00:00:00 to 2023-05-05 00:00:00
1 year growth rate (%) = 49.44325780125048 calculated from 2023-10-31 00:00:00 to 2022-11-06 00:00:00
3 years growth rate (%) = 986.9440892651639 calculated from 2023-10-31 00:00:00 to 2020-11-16 00:00:00


In [341]:
# Up/Down capture return
# Getting the monthly return of the Fund's AuM
fundReturns = getMonthlyReturns(f1_DailyAuM)
barclayReturns = barclayHedgeIndex("./Data/Barclay_Hedge_Fund_Indexhistorical_data.xls")
barclayReturns = barclayReturns[(fundReturns.iloc[0].Date <= barclayReturns.Date)&(barclayReturns.Date <= fundReturns.iloc[-1].Date)].reset_index(drop=True)

df = pd.concat([fundReturns.Date, fundReturns.fundReturn, barclayReturns.barclayReturn], axis = 1)
dfCumulativeReturn = calcCumReturnInRange(df, "months", 6)

# Up capture
upDf = dfCumulativeReturn[0 <= dfCumulativeReturn.barclayReturn]
upCapture = calcCaptureRate(upDf[["fundReturn", "barclayReturn"]])
# Down capture
dnDf = dfCumulativeReturn[dfCumulativeReturn.barclayReturn <= 0]
dnCapture = calcCaptureRate(dnDf[["fundReturn", "barclayReturn"]])

print(f"6 months:        up capture rate: {upCapture}   |    down capture rate: {dnCapture}")

6 months:        up capture rate: 9.752609614561386   |    down capture rate: -7.2632011214009795


In [343]:
dfCumulativeReturn

,intervalStart,intervalEnd,fundReturn,barclayReturn
0,2023-05-01,2023-10-31,-0.167980,0.017244
1,2022-11-01,2023-04-30,0.649983,0.058794
2,2022-05-01,2022-10-31,0.451293,-0.086267
3,2021-11-01,2022-04-30,0.195608,-0.016365
4,2021-05-01,2021-10-31,0.373640,0.037809
5,2020-11-01,2021-04-30,1.343934,0.145525
6,2020-05-01,2020-10-31,0.279866,0.153922
